## K-Means Clustering Algorithms
### ID: eo9232
### Name: Md Reza
### CSC 5825 - Fall 2021


In [1]:
# Import requires packages and libraries
import math
import pandas as pd
import csv
import numpy as np
import random

from scipy.sparse import random as sparse_random
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.random_projection import sparse_random_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations
from scipy.special import comb

##### Load the training and test data sets

In [2]:
def load_training_set():
    df_train = pd.read_csv("/u/mreza6/5825/Data/fashion-mnist_train.csv")
    return df_train

In [3]:
def load_test_set():
    df_test = pd.read_csv("/u/mreza6/5825/Data/fashion-mnist_test.csv")
    return df_test

In [4]:
train_data = load_training_set()
test_data = load_test_set()

##### Create training and test features where each row uses a vector of dimension 784 with values between 0 (black) and 255 (white) on the gray color scale

In [5]:
# Training
X_Training_features = train_data.iloc[:,1:785]
Y_Training_features = train_data.iloc[:,0]

# Test
X_Test_features =test_data.iloc[:,1:785]
Y_Test_features = test_data.iloc[:,0]

##### Test validation split

In [6]:
X_Train, X_Val, y_train, y_val = train_test_split(X_Training_features, Y_Training_features, test_size=0.10, random_state=42)

##### Scaling the input data

In [7]:
scaler = StandardScaler()
X_Train_features = scaler.fit_transform(X_Train)
X_Val=StandardScaler().fit_transform(X_Val.values)
X_Test = StandardScaler().fit_transform(X_Test_features.values)

##### Implement PCA for dimensionality reduction

In [8]:
pca = PCA(n_components=10)
train_pca= pca.fit(X_Train_features).transform(X_Train_features)
val_pca=pca.fit(X_Val).transform(X_Val)
test_pca = pca.fit(X_Test).transform(X_Test)

##### Implement the K-Means Algorithm

In [9]:
class K_Means:
    def __init__(self, k, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, data):

        self.centroids = {}
        # Initialize Random Centroid
        for i in range(self.k):
            r = np.random.randint(0, 50)
            self.centroids[i] = data[r]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self, data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

##### Define RAND Index Function. Given the true and predicted labels, it will return the RAND Index.

In [10]:
def rand_index(labels_true, labels_pred):
    my_pair = list(combinations(range(len(labels_true)), 2))

    def is_equal(x):
        return (x[0] == x[1])
    
    my_a = 0
    my_b = 0
    
    for i in range(len(my_pair)):
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == True):
            my_a += 1
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == False):
            my_b += 1
            
    my_denom = comb(len(labels_true), 2)
    RAND_index = (my_a + my_b) / my_denom
    return RAND_index

In [11]:
y_val=y_val.values.tolist()

##### Validate the results using the RAND index and adjusted RAND index with k values from 5 to 15 and summarize the results in a tabular format.

In [12]:
Best_K=0
Adjust_RAND=0

for k in range(5,16):
    number_of_clusters = k
    kmeans = K_Means(k)
    
    y_pred=[]    
    kmeans.fit(train_pca)
    for valp in val_pca:
        a=kmeans.predict(valp)
        y_pred.append(a)
   
    rand_score=rand_index(y_val,y_pred)
    scorerand=adjusted_rand_score(y_val, y_pred)
    
    data = [['K Value','RAND Index','Adjusted RAND Index'],[k,rand_score,scorerand]]
    print(tabulate(data, headers='firstrow'))
    print('\n')
    
    if(Adjust_RAND<rand_score):
        Adjust_RAND=rand_score
        Best_K=k
    
print('\n\nBest K:', Best_K)

  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
        5      0.759429               0.212436


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
        6      0.824444               0.256322


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
        7      0.787623               0.219692


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
        8      0.798864               0.229001


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
        9      0.850392               0.270137


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
       10      0.862149               0.284221


  K Value    RAND Index    Adjusted RAND Index
---------  ------------  ---------------------
       11       0.85618               0.286786



##### Run the algorithm 5 times over the Fashion-MNIST dataset with the best k value from the previous step, each time using a different initialization & report RAND index and adjusted RAND Score.

In [13]:
Y_Test_features=Y_Test_features.values.tolist()
for i in range(0,6):
    kmeans = K_Means(Best_K)
    kmeans.fit(train_pca)
    y_pred=[]
    
    for valt in test_pca:
        a=kmeans.predict(valt)
        y_pred.append(a)
    
    rand_score=rand_index(Y_Test_features,y_pred)
    scorerand=adjusted_rand_score(Y_Test_features, y_pred)
    
    data = [['Run','RAND Index','Adjusted RAND Index'],[i,rand_score,scorerand]]
    print(tabulate(data, headers='firstrow'))
    print('\n') 

  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    0      0.881036                 0.3399


  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    1      0.879178               0.331165


  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    2      0.873188               0.346019


  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    3      0.892339               0.385139


  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    4      0.870908               0.340228


  Run    RAND Index    Adjusted RAND Index
-----  ------------  ---------------------
    5      0.893311               0.385637




### Fndings During Experiments:

##### K-Means Clustering is relatively simple to implement and scales to large data sets, and easy to interpret the clustering results. It guarantees convergence & can warm-start the positions of centroids. Easily adapts to new examples. On the other hand, due to random choice of clustering, K-Means Clustering suffers from inconsistency. Handles only numerical data, & the number of clusters needs to be specified for effective clustering.